# Problem 3: Padding


## Overview

- Purpose: implement SHA-256 message padding and 512-bit block parsing per FIPS 180-4.
- Key functions: `append_terminator`, `pad_to_block_size`, `append_length`, `block_parse`.
- Workflow: start from a bytes message, append 0x80, zero‑pad until `(len(padded) + 8) % 64 == 0`, then append the 64‑bit big‑endian bit length and yield 64‑byte blocks.
- Notes: Produces blocks ready for the compression function; tests below cover empty input, "abc", and the 56‑byte edge case.

## SHA-256 Message Padding (Section 5.1.1 & 5.2.1)

**Padding rules:**
1. Append a single `1` bit (0x80 byte) immediately after the message
2. Append `k` zero bits where `k` is the smallest non-negative solution to:
   - `(message_length_bits + 1 + k + 64) ≡ 0 (mod 512)`
3. Append the 64-bit big-endian representation of the original message length in bits

**Result:** The padded message length is a multiple of 512 bits (64 bytes).

**Example:** 
- Message `"abc"` (24 bits) → pad with 1 bit, 423 zero bits, then 64-bit length = 512 bits total (1 block)
- Empty message (0 bits) → pad with 1 bit, 447 zero bits, then 64-bit length = 512 bits total (1 block)

## Helper Functions for Padding

Breaking down the padding process into modular steps for clarity and reusability.

## `append_terminator(padded)`

- **Purpose**: Append the '1' bit (0x80 byte) to mark the end of the message.
- **Args**: `padded` (bytearray) — the message being padded.
- **Modifies**: Appends 0x80 in-place.
- **FIPS Ref**: Section 5.1.1, step 1.

In [11]:
def append_terminator(padded):
    """
    Append the '1' bit (0x80 byte) to the message.
    """
    padded.append(0x80)


## `pad_to_block_size(padded)`

- **Purpose**: Append zero bytes until the message is ready for the 64-bit length field.
- **Args**: `padded` (bytearray) — the message with 0x80 already appended.
- **Modifies**: Appends zero bytes until `(len(padded) + 8) % 64 == 0`.
- **FIPS Ref**: Section 5.1.1, step 2.

In [12]:
def pad_to_block_size(padded):
    """
    Append zero bytes until room for the 64-bit length field is available.
    """
    while (len(padded) + 8) % 64 != 0:
        padded.append(0x00)


## `append_length(padded, msg_len_bits)`

- **Purpose**: Append the original message length in bits as a 64-bit big-endian integer.
- **Args**: 
  - `padded` (bytearray) — the padded message.
  - `msg_len_bits` (int) — original message length in bits.
- **Modifies**: Appends 8 bytes (64 bits) representing the message length.
- **FIPS Ref**: Section 5.1.1, step 3.

In [13]:
def append_length(padded, msg_len_bits):
    """
    Append the original message length in bits as a 64-bit big-endian integer.
    """
    padded.extend(msg_len_bits.to_bytes(8, byteorder='big'))

## `block_parse(msg)`

- Purpose: Generator function that parses a message into 512-bit (64-byte) blocks with proper SHA-256 padding.
- Args: `msg` (bytes) — the message to process.
- Yields: Each 512-bit block as a `bytes` object (64 bytes each).
- Padding: Follows FIPS 180-4 Section 5.1.1:
  1. Append 0x80 (single 1 bit followed by zeros)
  2. Append zero bytes until `(len + 9) % 64 == 0`
  3. Append original message length in bits as 64-bit big-endian integer
- Example: `list(block_parse(b"abc"))` yields one 64-byte block with proper padding.

In [14]:
def block_parse(msg):
    """
    Generator that yields 512-bit (64-byte) blocks from msg with SHA-256 padding.
    
    Implements FIPS 180-4 Section 5.1.1 (Padding) and 5.2.1 (Parsing).
    Uses helper functions to break padding into clear, modular steps.
    
    Args:
        msg (bytes): The message to process.
        
    Yields:
        bytes: Each 512-bit block (64 bytes).
    """
    if not isinstance(msg, bytes):
        raise TypeError("msg must be a bytes object")
    
    # Calculate original message length in bits
    msg_len_bits = len(msg) * 8
    
    # Start with the original message
    padded = bytearray(msg)
    
    # Apply padding steps
    append_terminator(padded)
    pad_to_block_size(padded)
    append_length(padded, msg_len_bits)
    
    # Yield 512-bit (64-byte) blocks
    for i in range(0, len(padded), 64):
        yield bytes(padded[i:i+64])


## Test Cases

Testing `block_parse()` with various message lengths to verify proper padding and block output.

In [15]:
def test_block_parse():    
    """
    Test the block_parse function with various message lengths.
    """
    # Test 1: Empty message (should produce 1 block)
    print("Test 1: Empty message")
    msg1 = b""
    blocks1 = list(block_parse(msg1))
    print(f"  Message length: {len(msg1)} bytes")
    print(f"  Number of blocks: {len(blocks1)}")
    print(f"  Block size: {len(blocks1[0])} bytes")
    print(f"  First block (hex): {blocks1[0].hex()}")
    print(f"  Last 8 bytes (length field): {blocks1[0][-8:].hex()} = {int.from_bytes(blocks1[0][-8:], 'big')} bits")
    print()

    # Test 2: "abc" (3 bytes, should produce 1 block)
    print("Test 2: Message 'abc'")
    msg2 = b"abc"
    blocks2 = list(block_parse(msg2))
    print(f"  Message length: {len(msg2)} bytes")
    print(f"  Number of blocks: {len(blocks2)}")
    print(f"  Block size: {len(blocks2[0])} bytes")
    print(f"  First 4 bytes: {blocks2[0][:4].hex()} (should be '61626380')")
    print(f"  Last 8 bytes (length field): {blocks2[0][-8:].hex()} = {int.from_bytes(blocks2[0][-8:], 'big')} bits")
    print()

    # Test 3: 56-byte message (should produce 2 blocks - just over the edge)
    print("Test 3: 56-byte message (requires 2 blocks)")
    msg3 = b"a" * 56
    blocks3 = list(block_parse(msg3))
    print(f"  Message length: {len(msg3)} bytes")
    print(f"  Number of blocks: {len(blocks3)}")
    print(f"  Block size of first block: {len(blocks3[0])} bytes")
    print(f"  Last 8 bytes of final block (length field): {blocks3[-1][-8:].hex()} = {int.from_bytes(blocks3[-1][-8:], 'big')} bits")
    print()

    print("All tests completed. Each block should be exactly 64 bytes.")

# Run the test function to validate block parsing.
test_block_parse()

Test 1: Empty message
  Message length: 0 bytes
  Number of blocks: 1
  Block size: 64 bytes
  First block (hex): 80000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000
  Last 8 bytes (length field): 0000000000000000 = 0 bits

Test 2: Message 'abc'
  Message length: 3 bytes
  Number of blocks: 1
  Block size: 64 bytes
  First 4 bytes: 61626380 (should be '61626380')
  Last 8 bytes (length field): 0000000000000018 = 24 bits

Test 3: 56-byte message (requires 2 blocks)
  Message length: 56 bytes
  Number of blocks: 2
  Block size of first block: 64 bytes
  Last 8 bytes of final block (length field): 00000000000001c0 = 448 bits

All tests completed. Each block should be exactly 64 bytes.
